# Stuart Landau Example 

This notebook contains the 1D-Stuart-Landau example from the MCBB paper. For all details please see the paper. The 2 2D-setups are in seperate files. 

For loading the saved jld files use the branch `old-state-sl-paper` of MCBB.

In [1]:
cluster = false
calc_distance = false
using Distributed
if cluster
    using ClusterManagers
    N_tasks = parse(Int, ARGS[1])
    task_type = parse(Int, ARGS[2])
    SAVE_NAME = ARGS[3]

    if task_type == 1
        N_worker = N_tasks
        addprocs(SlurmManager(N_worker))
        calc_mc = true
        calc_distance = false
    elseif task_type == 2
        calc_mc = false
        calc_distance = true
    else
        error("Unkown task_type, 1: Solve MCBB Problem, 2: Compute Distances")
    end
else
    SAVE_NAME = "stuart-landau-standard-larger-eps-range"
    import PyPlot
    using Plots, GraphPlot
    using LaTeXStrings

end

@everywhere using MCBB
using JLD2, Clustering, LinearAlgebra, SparseArrays
@everywhere using Parameters, Distributions, DifferentialEquations, StatsBase, LightGraphs





@everywhere struct stuart_landau_sathiyadevi_pars <: DEParameters
    lambda::Real
    omega::Real
    eps::Real
    N::Int
    P_1::Int
    P_2::Int
    A_1::AbstractArray
    A_2::AbstractArray
end

@everywhere function stuart_landau_sathiyadevi_pars(lambda::Real, omega::Real, eps::Real, N::Int, P_1::Int, P_2::Int, p_r::Number=0.04)
    P_1 = Int(round(r_1*N))
    P_2 = Int(round(r_2*N))

    A_1 = Bool.(adjacency_matrix(watts_strogatz(N, 2*P_1, p_r)))
    A_2 = Bool.(adjacency_matrix(watts_strogatz(N, 2*P_2, p_r)))

    stuart_landau_sathiyadevi_pars(lambda, omega, eps, N, P_1, P_2, A_1, A_2)
end


@everywhere function stuart_landau_sathiyadevi_pars(lambda::Real, omega::Real, eps::Real, N::Int, r_1::Real, r_2::Real)
    P_1 = Int(round(r_1*N))
    P_2 = Int(round(r_2*N))
    stuart_landau_sathiyadevi_pars(lambda, omega, eps, N, P_1, P_2)
end


@everywhere function stuart_landau_sathiyadevi!(dz,z,p::stuart_landau_sathiyadevi_pars,t)
    epsP1 = p.eps/(2*p.P_1)
    epsP2 = p.eps/(2*p.P_2)

    for i=1:p.N
        attr_coup = 0.
        for j=1:N
            if p.A_1[i,j]
               attr_coup += complex(real(z[j]) - real(z[i]),0)
            end
        end
        attr_coup *= epsP1

        repul_coup = 0.
        for j=1:p.N
            if p.A_2[i,j]
                repul_coup += complex(0., imag(z[j]) - imag(z[i]))
            end
        end
        repul_coup *= epsP2

        dz[i] = (complex(p.lambda,p.omega) - abs(z[i])^2)*z[i] + attr_coup - repul_coup
    end
end








@everywhere N = 100
@everywhere r_1 = 0.01
@everywhere r_2 = 0.22
@everywhere omega = 2.0
@everywhere lambda = 1.0
@everywhere epsval = 1.925
@everywhere pars = stuart_landau_sathiyadevi_pars(lambda, omega, epsval, N, r_1, r_2)

@everywhere epsilons = () -> rand(Uniform(1.8,2.1))
@everywhere p_r_gen = () -> rand(Uniform(0.,0.05))
@everywhere ic_dist = () -> complex(rand(Uniform(-1,1)),rand(Uniform(-1,1)))

@everywhere N_ics = 12000
@everywhere tail_frac = 0.7

#@everywhere par_vars = (:eps, epsilons)

@everywhere new_sls_pars(old_par::stuart_landau_sathiyadevi_pars; eps=2.02, p_r=0.0) = stuart_landau_sathiyadevi_pars(old_par.lambda, old_par.omega, eps, old_par.N, old_par.P_1, old_par.P_2, p_r)
#@everywhere par_vars = MultiDimParameterVar([OneDimParameterVar(:eps, epsilons),OneDimParameterVar(:p_r,p_r_gen)], new_sls_pars)

#@everywhere par_vars = (:p_r, p_r_gen, new_sls_pars)
@everywhere par_vars = (:eps, epsilons, new_sls_pars)
@everywhere odeprob = ODEProblem(stuart_landau_sathiyadevi!, rand(Complex{Float64},N), (0.,200.), pars)


@everywhere mean_real(arr::Array{Complex{Float64},1}, past_measures::AbstractArray) = mean(real.(arr))
@everywhere std_real(arr::Array{Complex{Float64},1}, past_measures::AbstractArray) = std(real.(arr); mean=past_measures[1])
@everywhere kl_real(arr::AbstractArray, past_measures::AbstractArray) = empirical_1D_KL_divergence_hist(real.(arr), past_measures)

@everywhere mean_imag(arr::Array{Complex{Float64},1}, past_measures::AbstractArray) = mean(imag.(arr))
@everywhere std_imag(arr::Array{Complex{Float64},1}, past_measures::AbstractArray) = std(imag.(arr); mean=past_measures[4])
@everywhere kl_imag(arr::AbstractArray, past_measures::AbstractArray) = empirical_1D_KL_divergence_hist(imag.(arr), [past_measures[4], past_measures[5]])


@everywhere function eval_ode_run_stuart(sol,i)
    (N_dim, __) = size(sol)
    state_filter = collect(1:N)
    eval_funcs = [mean_real, std_real, kl_real, mean_imag, std_imag, kl_imag]
    global_eval_funcs = []
    matrix_eval_funcs = [correlation_ecdf]
    eval_ode_run(sol, i, state_filter, eval_funcs, matrix_eval_funcs, global_eval_funcs, flag_past_measures=true)
end




















distance_func = (x,y)->sum(abs.(x.-y))

weights = [0.25, 1., 0., 0.25, 1., 0.,0., 1.]
K_BIN_FAC = 0.25

if cluster
    prob = DEMCBBProblem(odeprob, ic_dist, N_ics, pars, par_vars, eval_ode_run_stuart, tail_frac);
    println("solving now...")
    sol = solve(prob);
    println("finished!")
    @save string(SAVE_NAME,"-ps.jld2") prob sol
else
    @load string(SAVE_NAME,"-ps.jld2") prob sol

end


if calc_distance
    sol = normalize(sol)

    #println("calculating first distance matrix")
    #D = distance_matrix(sol, prob, [0.5, 1., 0.25, 0.5, 1., 0.25, 1.])
    #@save string(SAVE_NAME,"-d.jld2") D

    println("calculating second distance matrix")
    
    D_hist = distance_matrix(sol, prob, distance_func, weights, histograms=true, matrix_distance_func=MCBB.wasserstein_histogram_distance, k_bin=K_BIN_FAC)
    @save string(SAVE_NAME,"-d-hist-corr.jld2") D_hist
    
    # normalize sol for the distance computation but not for the later plots and so on. 
    @load string(SAVE_NAME,"-ps.jld2") prob sol

else
    if !(cluster)
        #@load string(SAVE_NAME,"-d.jld2") D
        @load string(SAVE_NAME,"-d-hist-corr.jld2") D_hist
    end
end

┌ Info: Recompiling stale cache file /home/max/.julia/compiled/v1.1/GraphPlot/bUwXr.ji for GraphPlot [a2cc645c-3eea-5389-862e-a155d0052231]
└ @ Base loading.jl:1184
┌ Info: Recompiling stale cache file /home/max/.julia/compiled/v1.1/MCBB/OqZ54.ji for MCBB [e96e44f0-e827-11e8-0ed4-0f2eba430555]
└ @ Base loading.jl:1184
┌ Warning: read type ODEFunction{true,typeof(stuart_landau_sathiyadevi!),UniformScaling{Bool},Nothing,Nothing,Nothing,Nothing,Nothing,Nothing,Nothing,Nothing,TCV} where TCV is not a leaf type in workspace; reconstructing
└ @ JLD2 /home/max/.julia/packages/JLD2/KjBIK/src/data.jl:477
┌ Warning: some parameters could not be resolved for type EnsembleProblem{ODEProblem{Array{Complex{Float64},1},Tuple{Float64,Float64},true,stuart_landau_sathiyadevi_pars,getfield(JLD2.ReconstructedTypes, Symbol("##ODEFunction{true,typeof(stuart_landau_sathiyadevi!),UniformScaling{Bool},Nothing,Nothing,Nothing,Nothing,Nothing,Nothing,Nothing,Nothing,TCV} where TCV#363")),Nothing,DiffEqBase.Stand

MethodError: MethodError: Cannot `convert` an object of type getfield(JLD2.ReconstructedTypes, Symbol("##EnsembleProblem{ODEProblem{Array{Complex{Float64},1},Tuple{Float64,Float64},true,stuart_landau_sathiyadevi_pars,getfield(JLD2.ReconstructedTypes, Symbol("##ODEFunction{true,typeof(stuart_landau_sathiyadevi!),UniformScaling{Bool},Nothing,Nothing,Nothing,Nothing,Nothing,Nothing,Nothing,Nothing,TCV} where TCV#363")),Nothing,DiffEqBase.StandardODEProblem},getfield(MCBB, Symbol("#new_problem#37")){Array{Complex{Float64},2},Array{Float64,2},stuart_landau_sathiyadevi_pars,Int64,Array{getfield(MCBB, Symbol("##39#40")){getfield(Main, Symbol("##7#8"))},1},ParameterVarFunc},typeof(eval_ode_run_stuart),DiffEqBase.##388#394,Array{Any,1}}#364")) to an object of type EnsembleProblem
Closest candidates are:
  convert(::Type{T}, !Matched::T) where T at essentials.jl:154
  EnsembleProblem(::T, !Matched::T2, !Matched::T3, !Matched::T4, !Matched::T5) where {T, T2, T3, T4, T5} at /home/max/.julia/packag

In [2]:
k = 4
fdist2 = k_dist(D_hist,k);
x = collect(1:sol.N_mc)

plot(x[1:end],fdist2[1:end])

UndefVarError: UndefVarError: D_hist not defined

# Clustering 1 

Coarse Resolution -> more fine structure 

In [3]:
db_eps = 0.008
db_res_1 = dbscan(D_hist,db_eps,10)
db_res_1.counts

UndefVarError: UndefVarError: D_hist not defined

### Names of the Clusters 

In [4]:
clabels = ["Outlier", "TW", "RCSW", "PS", "OD"];

In [5]:
Plots.pgfplots()
cluster_members_1 = cluster_membership(prob, db_res_1, 0.025, 0.01; min_members = 50);

Plots.plot(cluster_members_1, linecolor="white", guidefontsize=14, legendfontsize=12, fillalpha=0.6, xlabel=L"Coupling $K$", label=reverse(clabels))


┌ Info: Recompiling stale cache file /home/max/.julia/compiled/v1.1/PGFPlots/XxINN.ji for PGFPlots [3b7a836e-365b-5785-a47d-02c71176b4aa]
└ @ Base loading.jl:1184


UndefVarError: UndefVarError: prob not defined

In [6]:
sort!(cluster_members_1, ignore_first=true)
Plots.plot(cluster_members_1, linecolor="white", guidefontsize=14, legendfontsize=12, fillalpha=0.6, xlabel=L"Coupling $K$", label=reverse(clabels))


UndefVarError: UndefVarError: cluster_members_1 not defined

In [7]:
sort!(cluster_members_1, ignore_first=false)
Plots.plot(cluster_members_1, linecolor="white", guidefontsize=14, legendfontsize=12, fillalpha=0.6, xlabel=L"Coupling $K$", label=reverse(clabels))


UndefVarError: UndefVarError: cluster_members_1 not defined

In [8]:
?sort!

search: sort! partialsort! sortperm! partialsortperm! setproperty! showgradient!



```
sort!(v; alg::Algorithm=defalg(v), lt=isless, by=identity, rev::Bool=false, order::Ordering=Forward)
```

Sort the vector `v` in place. `QuickSort` is used by default for numeric arrays while `MergeSort` is used for other arrays. You can specify an algorithm to use via the `alg` keyword (see Sorting Algorithms for available algorithms). The `by` keyword lets you provide a function that will be applied to each element before comparison; the `lt` keyword allows providing a custom "less than" function; use `rev=true` to reverse the sorting order. These options are independent and can be used together in all possible combinations: if both `by` and `lt` are specified, the `lt` function is applied to the result of the `by` function; `rev=true` reverses whatever ordering specified via the `by` and `lt` keywords.

# Examples

```jldoctest
julia> v = [3, 1, 2]; sort!(v); v
3-element Array{Int64,1}:
 1
 2
 3

julia> v = [3, 1, 2]; sort!(v, rev = true); v
3-element Array{Int64,1}:
 3
 2
 1

julia> v = [(1, "c"), (3, "a"), (2, "b")]; sort!(v, by = x -> x[1]); v
3-element Array{Tuple{Int64,String},1}:
 (1, "c")
 (2, "b")
 (3, "a")

julia> v = [(1, "c"), (3, "a"), (2, "b")]; sort!(v, by = x -> x[2]); v
3-element Array{Tuple{Int64,String},1}:
 (3, "a")
 (2, "b")
 (1, "c")
```

---

```
sort!(A; dims::Integer, alg::Algorithm=defalg(v), lt=isless, by=identity, rev::Bool=false, order::Ordering=Forward)
```

Sort the multidimensional array `A` along dimension `dims`. See [`sort!`](@ref) for a description of possible keyword arguments.

To sort slices of an array, refer to [`sortslices`](@ref).

!!! compat "Julia 1.1"
    This function requires at least Julia 1.1.


# Examples

```jldoctest
julia> A = [4 3; 1 2]
2×2 Array{Int64,2}:
 4  3
 1  2

julia> sort!(A, dims = 1); A
2×2 Array{Int64,2}:
 1  2
 4  3

julia> sort!(A, dims = 2); A
2×2 Array{Int64,2}:
 1  2
 3  4
```

---

```
sort(sol::DEMCBBSol, prob::DEMCBBProblem, i::Int=1)
```

Sorts `sol` and `prob` inplace by the values of the `i`-th parameter.

(In case the parameter is complex valued, it sorts by the absolute value of the parameter)

---

```
sort(prob::DEMCBBProblem, i::Int=1)
```

Sorts `prob` inplace by the values of the`i`-th paramater.

(In case the parameter is complex valued, it sorts by the absolute value of the parameter)

---

```
sort!(cm::ClusterMembershipResult; ignore_first::Bool=false)
```

Sorts `cm` inplace by the count of members of the clusters from low to high. If `ignore_first` is true, the first cluster (with DBSCAN this is the outlier cluster) is ignored while sorting and remains the first cluster.

---

```
sort!(df::AbstractDataFrame, cols;
      alg::Union{Algorithm, Nothing}=nothing, lt=isless, by=identity,
      rev::Bool=false, order::Ordering=Forward)
```

Sort data frame `df` by column(s) `cols`. `cols` can be either a `Symbol` or `Integer` column index, or a tuple or vector of such indices.

If `alg` is `nothing` (the default), the most appropriate algorithm is chosen automatically among `TimSort`, `MergeSort` and `RadixSort` depending on the type of the sorting columns and on the number of rows in `df`. If `rev` is `true`, reverse sorting is performed. To enable reverse sorting only for some columns, pass `order(c, rev=true)` in `cols`, with `c` the corresponding column index (see example below). See other methods for a description of other keyword arguments.

# Examples

```jldoctest
julia> df = DataFrame(x = [3, 1, 2, 1], y = ["b", "c", "a", "b"])
4×2 DataFrame
│ Row │ x     │ y      │
│     │ Int64 │ String │
├─────┼───────┼────────┤
│ 1   │ 3     │ b      │
│ 2   │ 1     │ c      │
│ 3   │ 2     │ a      │
│ 4   │ 1     │ b      │

julia> sort!(df, :x)
4×2 DataFrame
│ Row │ x     │ y      │
│     │ Int64 │ String │
├─────┼───────┼────────┤
│ 1   │ 1     │ c      │
│ 2   │ 1     │ b      │
│ 3   │ 2     │ a      │
│ 4   │ 3     │ b      │

julia> sort!(df, (:x, :y))
4×2 DataFrame
│ Row │ x     │ y      │
│     │ Int64 │ String │
├─────┼───────┼────────┤
│ 1   │ 1     │ b      │
│ 2   │ 1     │ c      │
│ 3   │ 2     │ a      │
│ 4   │ 3     │ b      │

julia> sort!(df, (:x, :y), rev=true)
4×2 DataFrame
│ Row │ x     │ y      │
│     │ Int64 │ String │
├─────┼───────┼────────┤
│ 1   │ 3     │ b      │
│ 2   │ 2     │ a      │
│ 3   │ 1     │ c      │
│ 4   │ 1     │ b      │

julia> sort!(df, (:x, order(:y, rev=true)))
4×2 DataFrame
│ Row │ x     │ y      │
│     │ Int64 │ String │
├─────┼───────┼────────┤
│ 1   │ 1     │ c      │
│ 2   │ 1     │ b      │
│ 3   │ 2     │ a      │
│ 4   │ 3     │ b      │
```


In [9]:
Plots.savefig("sl-cm-fine.pdf")

ErrorException: No current plot/subplot

We dont plot clusters with less than 50 members. The most important clusters have the following indices: 


In [10]:
plot_index_c1 = [1; findall(db_res_1.counts .> 50) .+ 1] # outlier cluster + the others 

UndefVarError: UndefVarError: db_res_1 not defined

## Clustering 2 

More coarser resolutions leeds to Part S / OD joining 

In [11]:
db_eps = 0.009
db_res_2 = dbscan(D_hist,db_eps,10)
db_res_2.counts

UndefVarError: UndefVarError: D_hist not defined

In [12]:
Plots.pgfplots()
cluster_members_2 = cluster_membership(prob, db_res_2, 0.025, 0.01; min_members = 90);

Plots.plot(cluster_members_2, linecolor="white", guidefontsize=14, legendfontsize=12, fillalpha=0.6, xlabel=L"Coupling $K$", label=["OD+PS","Part. S","SAC","TW","Outlier"])


UndefVarError: UndefVarError: prob not defined

In [13]:
Plots.savefig("sl-cm-coarse.pdf")

ErrorException: No current plot/subplot

# Backup: Even Finer 

We can resolve the transition 



In [14]:
db_eps = 0.008
db_res_3 = dbscan(D_hist,db_eps,20)
db_res_3.counts

UndefVarError: UndefVarError: D_hist not defined

In [15]:
Plots.pgfplots()
cluster_members_3 = cluster_membership(prob, db_res_3, 0.025, 0.01; min_members = 50);

Plots.plot(cluster_members_3, linecolor="white", fillalpha=0.6, guidefontsize= 14, legendfontsize=12, xlabel=L"Coupling $K$", label=["OD","Part. S","SAC","TW","Outlier"])


UndefVarError: UndefVarError: prob not defined

# Cluster Measure Histograms 

We do this with the finer resolustion of clustering 1

First, the real part means. (Generally real and imaginary parts are very similar)


In [16]:
 shists = cluster_measures_sliding_histograms(prob, sol, db_res_1, 1, 0.025, 0.01; k_bin=2.);

UndefVarError: UndefVarError: prob not defined

In [17]:
l = @layout [a; b; c; d; e]
Plots.pyplot()
# double check: titles/clusters could change orders 
p1 = Plots.plot(shists, plot_index_c1[1], c=:viridis, title=string(L"$\mu(Re(x_i))$ of ", clabels[1] ," Cluster"), xlabel="Coupling K", ylabel="Value of Std")
p2 = Plots.plot(shists, plot_index_c1[2], c=:viridis, title=string(L"$\mu(Re(x_i))$ of ", clabels[2] ," Cluster"), xlabel="Coupling K", ylabel="Value of Std")
p3 = Plots.plot(shists, plot_index_c1[3], c=:viridis, title=string(L"$\mu(Re(x_i))$ of ", clabels[3] ," Cluster"), xlabel="Coupling K", ylabel="Value of Std")
p4 = Plots.plot(shists, plot_index_c1[4], c=:viridis, title=string(L"$\mu(Re(x_i))$ of ", clabels[4] ," Cluster"), xlabel="Coupling K", ylabel="Value of Std")
p5 = Plots.plot(shists, plot_index_c1[5], c=:viridis, title=string(L"$\mu(Re(x_i))$ of ", clabels[5] ," Cluster"), xlabel="Coupling K", ylabel="Value of Std")


Plots.plot(p1, p2, p3, p4, p5, layout = l, size=(400,1200))

UndefVarError: UndefVarError: plot_index_c1 not defined

In [18]:
Plots.savefig("sl-hists-mean.pdf")

ErrorException: No current plot/subplot

Now, the Stds

In [19]:
l = @layout [a; b; c; d; e]

shists = cluster_measures_sliding_histograms(prob, sol, db_res_1, 2, 0.025, 0.01; k_bin=2.);
# double check: titles/clusters could change orders 
p1 = Plots.plot(shists, plot_index_c1[1], c=:viridis, title=string(L"$\sigma(Re(x_i))$ of ", clabels[1] ," Cluster"), xlabel="Coupling K", ylabel="Value of Std")
p2 = Plots.plot(shists, plot_index_c1[2], c=:viridis, title=string(L"$\sigma(Re(x_i))$ of ", clabels[2] ," Cluster"), xlabel="Coupling K", ylabel="Value of Std")
p3 = Plots.plot(shists, plot_index_c1[3], c=:viridis, title=string(L"$\sigma(Re(x_i))$ of ", clabels[3] ," Cluster"), xlabel="Coupling K", ylabel="Value of Std")
p4 = Plots.plot(shists, plot_index_c1[4], c=:viridis, title=string(L"$\sigma(Re(x_i))$ of ", clabels[4] ," Cluster"), xlabel="Coupling K", ylabel="Value of Std")
p5 = Plots.plot(shists, plot_index_c1[5], c=:viridis, title=string(L"$\sigma(Re(x_i))$ of ", clabels[5] ," Cluster"), xlabel="Coupling K", ylabel="Value of Std")


Plots.plot(p1, p2, p3, p4, p5, layout = l, size=(400,1200))

UndefVarError: UndefVarError: prob not defined

In [20]:
Plots.savefig("sl-hists-std.pdf")

ErrorException: No current plot/subplot

Next, we ll look at the average measures for each cluster and its std

In [21]:
colororder = ["blue","orange","green","red","purple","brown","magenta","olive","cyan"]
measure_1 = 1
measure_2 = 2

m1m = cluster_measure_mean(sol, db_res_1, measure_1)[plot_index_c1]
m1sd = cluster_measure_std(sol, db_res_1, measure_1)[plot_index_c1]
m2m = cluster_measure_mean(sol, db_res_1, measure_2)[plot_index_c1]
m2sd = cluster_measure_std(sol, db_res_1, measure_2)[plot_index_c1]

SC = PyPlot.scatter(m1m, m2m, c=colororder[1:length(m1m)])
#PyPlot.legend(SC,["Noise","2","3","4","5","6","7","8","9","10","11"])

PyPlot.errorbar(m1m[1], m2m[1], fmt="o", ms=0,ecolor=colororder[1], xerr=0.005*m1sd[1], yerr=m2sd[1], label=clabels[1])
PyPlot.errorbar(m1m[2], m2m[2], fmt="o", ms=0,ecolor=colororder[2], xerr=0.005*m1sd[2], yerr=m2sd[2], label=clabels[2])
PyPlot.errorbar(m1m[3], m2m[3], fmt="o", ms=0,ecolor=colororder[3], xerr=0.005*m1sd[3], yerr=m2sd[3], label=clabels[3])
PyPlot.errorbar(m1m[4], m2m[4], fmt="o", ms=0,ecolor=colororder[4], xerr=0.005*m1sd[4], yerr=m2sd[4], label=clabels[4])
PyPlot.errorbar(m1m[5], m2m[5], fmt="o", ms=0,ecolor=colororder[5], xerr=0.005*m1sd[5], yerr=m2sd[5], label=clabels[5])

PyPlot.legend()
PyPlot.xlabel("Average Mean")
PyPlot.ylabel("Average Std")

UndefVarError: UndefVarError: sol not defined

# Example Trajectories 

In [22]:
?get_trajectory

search: get_trajectory



```
get_trajectory(prob::MCBBProblem, sol::MCBBSol, clusters::ClusteringResult, i::Int; only_sol::Bool=true)
```

Solves and returns a trajectory that is classified in cluster `i`. Randomly selects one IC/Parameter configuration, so that mulitple executions of this routine will yield different results! If `only_sol==true` it returns only the solution, otherwise it returns a tuple `(solution, problem, i_run)` where `i_run` is the number of the trial in `prob` and `sol`.

```
get_trajectory(prob::MCBBProblem, sol::MCBBSol, i::Int, only_sol::Bool=true)
```

Solves problem `i` and returns a trajectory. If `only_sol==true` it returns only the solution, otherwise it returns a tuple `(solution, problem, i_run)` where `i_run` is the number of the trial in `prob` and `sol`.

# Example

Plot with e.g

```
using PyPlot
IM = imshow(Matrix(get_trajectory(prob,sol,db_res,1)), aspect=2)
ylabel("System Dimension i")
xlabel("Time t")
cb = colorbar(IM, orientation="horizontal")
cb[:ax][:set_xlabel]("Colorbar: Im(z)", rotation=0)
```


In [23]:
traj1 = get_trajectory(prob, sol, db_res_1, plot_index_c1[2])
IM = PyPlot.imshow(Matrix(imag.(traj1)))
PyPlot.ylabel(L"Oscillator $i$",fontsize=20) 
PyPlot.xlabel(L"Time $t$",fontsize=20)
PyPlot.title(string("Example Trajectory - ",clabels[2]), fontsize=20)
cb = PyPlot.colorbar(IM, orientation="horizontal")
cb[:ax][:set_ylabel]("Im(z)", fontsize=20)
PyPlot.savefig("traj-tw.pdf")

UndefVarError: UndefVarError: plot_index_c1 not defined

In [24]:
traj2 = get_trajectory(prob, sol, db_res_1, plot_index_c1[3])
IM = PyPlot.imshow(Matrix(imag.(traj2)))
PyPlot.ylabel(L"Oscillator $i$",fontsize=20) 
PyPlot.xlabel(L"Time $t$",fontsize=20)
PyPlot.title(string("Example Trajectory - ",clabels[3]), fontsize=20)
cb = PyPlot.colorbar(IM, orientation="horizontal")
cb[:ax][:set_ylabel]("Im(z)", fontsize=20)
PyPlot.savefig("traj-rcsw.pdf")
#savefig("trajectory-ac.pdf")

UndefVarError: UndefVarError: plot_index_c1 not defined

In [25]:
traj3 = get_trajectory(prob, sol, db_res_1, plot_index_c1[4])
IM = PyPlot.imshow(Matrix(imag.(traj3)))
PyPlot.ylabel(L"Oscillator $i$",fontsize=20) 
PyPlot.xlabel(L"Time $t$",fontsize=20)
PyPlot.title(string("Example Trajectory - ",clabels[4]), fontsize=20)
cb = PyPlot.colorbar(IM, orientation="horizontal")
cb[:ax][:set_ylabel]("Im(z)", fontsize=20)
PyPlot.savefig("traj-ps.pdf")


UndefVarError: UndefVarError: plot_index_c1 not defined

In [26]:
traj4 = get_trajectory(prob, sol, db_res_1, plot_index_c1[5])
IM = PyPlot.imshow(Matrix(imag.(traj4)))
PyPlot.ylabel(L"Oscillator $i$",fontsize=20) 
PyPlot.xlabel(L"Time $t$",fontsize=20)
PyPlot.title(string("Example Trajectory - ",clabels[5]), fontsize=20)
cb = PyPlot.colorbar(IM, orientation="horizontal")
cb[:ax][:set_ylabel]("Im(z)", fontsize=20)
PyPlot.savefig("traj-od.pdf")

UndefVarError: UndefVarError: plot_index_c1 not defined

In [27]:
Plots.savefig("traj-od.pdf")

ErrorException: No current plot/subplot

## Cluster Measures

(not in the paper / or only in appendix)

In [28]:
cmr = cluster_measures(prob, sol, db_res_1, 0.025, 0.01);

UndefVarError: UndefVarError: prob not defined

In [29]:
#fig = figure(figsize=(10,10))
xlimits = [1.8,2.6];
ylimits_means = [-1,1];
plotis = []
for (i,j) in enumerate(plot_index_c1)
    #subplot(4,2,i)
   # plot(cmr.par, collect(cmr.cluster_measures[i,6,:,:]'), alpha=0.5);
    #plot(p_win, collect(cluster_measures_global[i,1,:,:]'), alpha=0.5);
    push!(plotis, Plots.plot(cmr, 1, j, xlims=xlimits, ylims=ylimits_means, title=clabels[i], ylabel="Mean", xlabel="Coupling K"))
    #xlim(xlimits[1],xlimits[2]);
    #ylim(ylimits_means[1],ylimits_means[2]);
end


Plots.plot(plotis...)


UndefVarError: UndefVarError: plot_index_c1 not defined

In [30]:
Plots.savefig("sl-1d-cl-means.pdf")

ErrorException: No current plot/subplot

In [31]:
#fig = figure(figsize=(10,10))
xlimits = [1.8,2.6];
ylimits_means = [0,1.1];
plotis = []
for (j,i) in enumerate(plot_index_c1)
    #subplot(4,2,i)
   # plot(cmr.par, collect(cmr.cluster_measures[i,6,:,:]'), alpha=0.5);
    #plot(p_win, collect(cluster_measures_global[i,1,:,:]'), alpha=0.5);
    push!(plotis, Plots.plot(cmr, 2, i, xlims=xlimits, ylims=ylimits_means, title=clabels[j], ylabel="Std", xlabel="Coupling K"))
    #xlim(xlimits[1],xlimits[2]);
    #ylim(ylimits_means[1],ylimits_means[2]);
end


Plots.plot(plotis...)
Plots.savefig("sl-1d-cl-std.pdf")

UndefVarError: UndefVarError: plot_index_c1 not defined

In [32]:
Plots.savefig("sl-1d-cl-std.pdf")

ErrorException: No current plot/subplot